# WaDi A1 - Pipeline Notebook 4: Feature Engineering  
* **Input:** Warehouse parquet from Notebook 3
* **Scope:** Computes rolling time-window for each sensor, assembles feature matrix, fits normalization on train split, writes final feature matrix.
* **Output:** Normalized feature matrix parquet, scaler artifact.

# Stage 0 - Setup

## 0.1 - Imports and Paths

In [1]:
from __future__ import annotations

from pathlib import Path
from datetime import datetime, timezone
import json
import joblib

import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from IPython.display import display

pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 180)

# Paths 
WORK_DIR     = Path("work")
PROJECT_DIR  = WORK_DIR / "wadi_A1"
DATA_DIR     = PROJECT_DIR / "data"
WH_DIR       = DATA_DIR / "warehouse"
FEATURES_DIR = DATA_DIR / "features"
REF_DIR      = DATA_DIR / "reference"
RUN_DIR      = REF_DIR / "pipeline_runs"

for p in [FEATURES_DIR, RUN_DIR]:
    p.mkdir(parents=True, exist_ok=True)

print("Project:   ", PROJECT_DIR)
print("Warehouse: ", WH_DIR)
print("Features:  ", FEATURES_DIR)
print("Reference: ", REF_DIR)

Project:    work/wadi_A1
Warehouse:  work/wadi_A1/data/warehouse
Features:   work/wadi_A1/data/features
Reference:  work/wadi_A1/data/reference


## 0.2 - Helper Utilities

In [2]:
class PipelineError(RuntimeError):
    pass

def utc_now_iso() -> str:
    return datetime.now(timezone.utc).isoformat()

def write_json(path: Path, obj: dict) -> None:
    path.parent.mkdir(parents=True, exist_ok=True)
    path.write_text(json.dumps(obj, indent=2, default=str))

def read_json(path: Path) -> dict:
    return json.loads(path.read_text())

print("Helpers ready.")

Helpers ready.


## 0.3 - Configuration

In [3]:
DATASET_NAME  = "WaDi.A1_9 Oct 2017"

# Rolling window configuration 
WINDOW_SIZE   = 60      # seconds — matches 1-second sampling rate directly
MIN_PERIODS   = 30      # minimum observations required to compute a valid statistic
                        # set to 50% of window to handle split boundaries cleanly

# Feature statistics 
ROLLING_STATS = ["mean", "std", "min", "max", "roc"]   # roc = rate of change

# Run ID 
RUN_ID = datetime.now(timezone.utc).strftime("%Y%m%d_%H%M%S_utc")

print(f"Dataset:      {DATASET_NAME}")
print(f"Window size:  {WINDOW_SIZE} seconds")
print(f"Min periods:  {MIN_PERIODS}")
print(f"Statistics:   {ROLLING_STATS}")
print(f"Run ID:       {RUN_ID}")

Dataset:      WaDi.A1_9 Oct 2017
Window size:  60 seconds
Min periods:  30
Statistics:   ['mean', 'std', 'min', 'max', 'roc']
Run ID:       20260223_142554_utc


# Stage 1 - Load Warehouse Data  
Loads the warehouse parquet from Notebook 3 and the canonical SENSOR_COLS reference from Notebook 1

## 1.1 - Load Warehouse Parquet

In [4]:
# Load most recent warehouse parquet
wh_files = sorted(WH_DIR.glob("wadi_curated_*.parquet"))
if not wh_files:
    raise PipelineError(f"No warehouse parquet found in {WH_DIR}")

wh_path = wh_files[-1]
print(f"Loading: {wh_path}")

df = pd.read_parquet(wh_path)
print(f"Shape:   {df.shape}")
print(f"\nLabel distribution:")
for label_val, label_name in [(0, "normal"), (1, "attack"), (2, "fault")]:
    n   = (df["label"] == label_val).sum()
    pct = n / len(df) * 100
    print(f"  {label_name:<8} ({label_val}): {n:>9,}  ({pct:.2f}%)")

print(f"\nSplit distribution:")
for split in ["train", "test"]:
    n = (df["split"] == split).sum()
    print(f"  {split:<6}: {n:>9,}")

Loading: work/wadi_A1/data/warehouse/wadi_curated_20260223_142029_utc.parquet
Shape:   (1382402, 103)

Label distribution:
  normal   (0): 1,116,251  (80.75%)
  attack   (1):   172,801  (12.50%)
  fault    (2):    93,350  (6.75%)

Split distribution:
  train : 1,209,601
  test  :   172,801


## 1.2 - Load Sensor Column List

In [5]:
# Load canonical sensor column list
sensor_cols_path = REF_DIR / "sensor_cols.json"
if not sensor_cols_path.exists():
    raise PipelineError(f"sensor_cols.json not found at {sensor_cols_path}")

sensor_ref  = read_json(sensor_cols_path)
SENSOR_COLS = sensor_ref["sensor_cols"]

print(f"SENSOR_COLS loaded: {len(SENSOR_COLS)} columns")
print(f"Source run ID:      {sensor_ref['run_id']}")

missing = [c for c in SENSOR_COLS if c not in df.columns]
if missing:
    raise PipelineError(f"SENSOR_COLS missing from warehouse data: {missing}")

print("All SENSOR_COLS present in warehouse data. ✓")

SENSOR_COLS loaded: 98 columns
Source run ID:      20260223_140105_utc
All SENSOR_COLS present in warehouse data. ✓


## 1.3 - Summary

In [6]:
print(f"Time range: {df['timestamp'].min()} → {df['timestamp'].max()}")
print(f"Columns:    {df.columns.tolist()[:8]} ... ({len(df.columns)} total)")
print(f"\nDtype summary:")
dtype_counts = df.dtypes.value_counts()
for dtype, count in dtype_counts.items():
    print(f"  {str(dtype):<15s}  {count} columns")

Time range: 2017-09-25 18:00:00+00:00 → 2017-10-11 18:00:00+00:00
Columns:    ['timestamp', 'observation_day', 'seconds_since_start', 'split', 'label', '1_AIT_001_PV', '1_AIT_002_PV', '1_AIT_003_PV'] ... (103 total)

Dtype summary:
  float32          99 columns
  object           2 columns
  datetime64[ns, UTC]  1 columns
  int8             1 columns


# Stage 2 - Compute Rolling Features  
Computes five rolling statistics for each sensor over a 60-second backward-looking window:  
* mean
* std
* min
* max
* rate of change

Features are computed per split independently to prevent any future data from crossing split boundaries

## 2.1 - Define Rolling Feature Functions

In [7]:
def compute_rolling_features(
    df_split: pd.DataFrame,
    sensor_cols: list[str],
    window: int,
    min_periods: int,
) -> pd.DataFrame:
    """
    Computes rolling statistics for each sensor column over a backward-looking
    window. Operates on a single split to prevent cross-split leakage.
    Returns a DataFrame of feature columns only, aligned to df_split.index.
    """
    feature_frames = []

    for col in sensor_cols:
        series = df_split[col].astype("float64")
        roll   = series.rolling(window=window, min_periods=min_periods)

        mean = roll.mean().rename(f"{col}__mean_{window}s")
        std  = roll.std().rename(f"{col}__std_{window}s")
        mn   = roll.min().rename(f"{col}__min_{window}s")
        mx   = roll.max().rename(f"{col}__max_{window}s")

        # Rate of change: (last - first) / window in seconds
        # raw=True passes numpy array — use [-1] and [0] not iloc
        roc = (
            roll.apply(lambda x: (x[-1] - x[0]) / window, raw=True)
        ).rename(f"{col}__roc_{window}s")

        feature_frames.extend([mean, std, mn, mx, roc])

    return pd.concat(feature_frames, axis=1).astype("float32")

print("Rolling feature function defined.")

Rolling feature function defined.


## 2.2 - Verify Rolling Features on Train Split  
Computes rolling features on the train split only first to verify output shape, NaN counts, and feature naming before running the full dataset

In [8]:
# Extract train split sorted by timestamp
df_train = df[df["split"] == "train"].sort_values("timestamp").reset_index(drop=True)

print(f"Train split shape: {df_train.shape}")
print(f"Computing rolling features on train split...")

train_features = compute_rolling_features(
    df_split    = df_train,
    sensor_cols = SENSOR_COLS,
    window      = WINDOW_SIZE,
    min_periods = MIN_PERIODS,
)

print(f"\nRolling feature shape: {train_features.shape}")
print(f"Expected:              ({len(df_train)}, {len(SENSOR_COLS) * len(ROLLING_STATS)})")

# NaN summary
n_nan = train_features.isna().sum().sum()
pct_nan = n_nan / train_features.size * 100
print(f"\nNaN values:  {n_nan:,}  ({pct_nan:.2f}%)")
print(f"(Expected: NaNs only in first {MIN_PERIODS} rows per sensor)")

# Sample feature names
print(f"\nSample feature names:")
print(f"  {train_features.columns[:5].tolist()}")
print(f"  ...")
print(f"  {train_features.columns[-5:].tolist()}")

Train split shape: (1209601, 103)
Computing rolling features on train split...

Rolling feature shape: (1209601, 490)
Expected:              (1209601, 490)

NaN values:  49,126  (0.01%)
(Expected: NaNs only in first 30 rows per sensor)

Sample feature names:
  ['1_AIT_001_PV__mean_60s', '1_AIT_001_PV__std_60s', '1_AIT_001_PV__min_60s', '1_AIT_001_PV__max_60s', '1_AIT_001_PV__roc_60s']
  ...
  ['TOTAL_CONS_REQUIRED_FLOW__mean_60s', 'TOTAL_CONS_REQUIRED_FLOW__std_60s', 'TOTAL_CONS_REQUIRED_FLOW__min_60s', 'TOTAL_CONS_REQUIRED_FLOW__max_60s', 'TOTAL_CONS_REQUIRED_FLOW__roc_60s']


## 2.3 - Compute Rolling Features on Full Dataset  
Computes rolling features per split independently to prevent cross-split leakage. Each split is sorted by timestamp, features computed, then recombined.

In [9]:
import gc

feature_splits = []

for split in ["train", "test"]:
    print(f"Computing rolling features for {split} split...")

    df_split = df[df["split"] == split].sort_values("timestamp").reset_index(drop=True)

    rolling_features = compute_rolling_features(
        df_split    = df_split,
        sensor_cols = SENSOR_COLS,
        window      = WINDOW_SIZE,
        min_periods = MIN_PERIODS,
    )

    df_combined = pd.concat([df_split, rolling_features], axis=1)
    del rolling_features
    gc.collect()

    feature_splits.append(df_combined)
    print(f"  Shape: {df_combined.shape}")
    del df_split, df_combined
    gc.collect()

# Final concat
df_features = pd.concat(feature_splits, ignore_index=True)
del feature_splits
gc.collect()

df_features = df_features.sort_values("timestamp").reset_index(drop=True)

print(f"\nFull dataset shape: {df_features.shape}")
print(f"Expected columns:   {5 + len(SENSOR_COLS) + len(SENSOR_COLS) * len(ROLLING_STATS)}")

Computing rolling features for train split...
  Shape: (1209601, 593)
Computing rolling features for test split...
  Shape: (172801, 593)

Full dataset shape: (1382402, 593)
Expected columns:   593


## 2.4 - Handle NaN Values at Split Boundaries  
Rolling windows produce NaN values in the first `MIN_PERIODS` rows of each split. These are filled with the column mean computed from the train split only. 

In [10]:
# Identify rolling feature columns
rolling_cols = [c for c in df_features.columns
                if any(f"__{stat}_" in c for stat in ROLLING_STATS)]

print(f"Rolling feature columns: {len(rolling_cols)}")

# Count NaNs before fill
n_nan_before = df_features[rolling_cols].isna().sum().sum()
print(f"NaN values before fill:  {n_nan_before:,}")

# Compute fill values from train split only
train_means = df_features[df_features["split"] == "train"][rolling_cols].mean()

# Fill NaNs with train means
df_features[rolling_cols] = df_features[rolling_cols].fillna(train_means)

# Count NaNs after fill
n_nan_after = df_features[rolling_cols].isna().sum().sum()
print(f"NaN values after fill:   {n_nan_after:,}")

# Handle any remaining NaNs in raw sensor columns
raw_nan = df_features[SENSOR_COLS].isna().sum().sum()
print(f"NaN values in raw sensor columns: {raw_nan:,}")
if raw_nan > 0:
    train_sensor_means = df_features[
        df_features["split"] == "train"
    ][SENSOR_COLS].mean()
    df_features[SENSOR_COLS] = df_features[SENSOR_COLS].fillna(train_sensor_means)
    print(f"Raw sensor NaNs filled with train means.")

Rolling feature columns: 490
NaN values before fill:  63,336
NaN values after fill:   0
NaN values in raw sensor columns: 10,279
Raw sensor NaNs filled with train means.


# Stage 3 - Assemble Feature Matrix  
* Defines the final feature column list
* Drops non-feature columns
* Confirms no NaN or inf values remain
* Freezes the `FEATURE_COLS` reference

## 3.1 - Define Feature Columns

In [11]:
# Feature matrix = raw sensor readings + rolling features
# Explicitly excludes: timestamp, observation_day, seconds_since_start
FEATURE_COLS = SENSOR_COLS + rolling_cols

print(f"Raw sensor features:     {len(SENSOR_COLS)}")
print(f"Rolling features:        {len(rolling_cols)}")
print(f"Total feature columns:   {len(FEATURE_COLS)}")
print(f"\nMetadata columns retained: split, label")
print(f"Columns excluded from feature matrix:")
print(f"  timestamp, observation_day, seconds_since_start")

Raw sensor features:     98
Rolling features:        490
Total feature columns:   588

Metadata columns retained: split, label
Columns excluded from feature matrix:
  timestamp, observation_day, seconds_since_start


## 3.2 - Confirm No NaN or Inf Values

In [12]:
# Check for NaN
n_nan = df_features[FEATURE_COLS].isna().sum().sum()
print(f"NaN values in feature matrix:  {n_nan:,}")

# Check for inf
n_inf = np.isinf(df_features[FEATURE_COLS].values).sum()
print(f"Inf values in feature matrix:  {n_inf:,}")

if n_nan > 0 or n_inf > 0:
    raise PipelineError(
        f"Feature matrix contains {n_nan} NaN and {n_inf} inf values — "
        "resolve before proceeding."
    )

print("\nFeature matrix is clean.")

NaN values in feature matrix:  0
Inf values in feature matrix:  0

Feature matrix is clean.


## 3.3 - Freeze `FEATURE_COLS` Reference

In [13]:
feature_cols_path = REF_DIR / "feature_cols.json"
write_json(feature_cols_path, {
    "run_id":           RUN_ID,
    "dataset":          DATASET_NAME,
    "window_size":      WINDOW_SIZE,
    "min_periods":      MIN_PERIODS,
    "rolling_stats":    ROLLING_STATS,
    "n_raw_features":   len(SENSOR_COLS),
    "n_rolling_features": len(rolling_cols),
    "n_total_features": len(FEATURE_COLS),
    "feature_cols":     FEATURE_COLS,
    "notes": [
        "Feature matrix excludes timestamp, observation_day, seconds_since_start.",
        "Rolling features computed per split independently — no cross-split leakage.",
        "NaN values filled with train split column means.",
        "Normalization applied in Stage 4 — scaler fit on train split only.",
    ]
})

print(f"FEATURE_COLS written to: {feature_cols_path}")
print(f"Total features frozen:   {len(FEATURE_COLS)}")


FEATURE_COLS written to: work/wadi_A1/data/reference/feature_cols.json
Total features frozen:   588


# Stage 4 - Normalize
Fits a StandardScaler on the train split only, then applies it to the test split.
This closes the normalization leakage risk identified in the Notebook 3 leakage audit.

## 4.1 - Fit Scaler on Train Split Only

In [14]:
train_mask = df_features["split"] == "train"

# Fit scaler in batches to avoid materializing full numpy copy
from sklearn.preprocessing import StandardScaler
import gc

# Use a sample for zero-variance detection (much cheaper)
# 100k rows is sufficient to detect zero-variance sensors
sample_idx = df_features.index[train_mask][:100_000]
X_sample = df_features.loc[sample_idx, FEATURE_COLS].values

scaler_initial = StandardScaler()
scaler_initial.fit(X_sample)
del X_sample
gc.collect()

# Identify zero-variance base sensors
zero_std_base_sensors = set()
for i, std in enumerate(scaler_initial.scale_):
    if std < 1e-6:
        base = FEATURE_COLS[i].split("__")[0]
        zero_std_base_sensors.add(base)

print(f"Zero-variance base sensors detected ({len(zero_std_base_sensors)}):")
for s in zero_std_base_sensors:
    print(f"  {s}")

# Drop ALL features belonging to zero-variance sensors
zero_std_cols = [
    c for c in FEATURE_COLS
    if c.split("__")[0] in zero_std_base_sensors
]
print(f"\nDropping {len(zero_std_cols)} features (raw + all rolling variants):")
for c in zero_std_cols:
    print(f"  {c}")

FEATURE_COLS = [c for c in FEATURE_COLS if c not in zero_std_cols]
print(f"\nFEATURE_COLS updated: {len(FEATURE_COLS)} features")

Zero-variance base sensors detected (3):
  2_LS_401_AH
  2_LS_201_AH
  3_AIT_001_PV

Dropping 18 features (raw + all rolling variants):
  2_LS_201_AH
  2_LS_401_AH
  3_AIT_001_PV
  2_LS_201_AH__mean_60s
  2_LS_201_AH__std_60s
  2_LS_201_AH__min_60s
  2_LS_201_AH__max_60s
  2_LS_201_AH__roc_60s
  2_LS_401_AH__mean_60s
  2_LS_401_AH__std_60s
  2_LS_401_AH__min_60s
  2_LS_401_AH__max_60s
  2_LS_401_AH__roc_60s
  3_AIT_001_PV__mean_60s
  3_AIT_001_PV__std_60s
  3_AIT_001_PV__min_60s
  3_AIT_001_PV__max_60s
  3_AIT_001_PV__roc_60s

FEATURE_COLS updated: 570 features


## 4.2 - Fit Final Scaler On Cleaned Features

In [15]:
# 4.2 - Fit final scaler in chunks
from sklearn.preprocessing import StandardScaler

CHUNK_ROWS = 50_000
scaler = StandardScaler()

train_indices = df_features.index[train_mask].tolist()

for i in range(0, len(train_indices), CHUNK_ROWS):
    chunk_idx = train_indices[i:i + CHUNK_ROWS]
    X_chunk = df_features.loc[chunk_idx, FEATURE_COLS].values
    scaler.partial_fit(X_chunk)
    del X_chunk
    gc.collect()

print(f"Scaler fit on train split: {len(train_indices):,} rows")
print(f"Features:                  {len(FEATURE_COLS)}")

Scaler fit on train split: 1,209,601 rows
Features:                  570


## 4.3 - Apply Scaler to Full Dataset

In [16]:
# 4.3 - Apply scaler in chunks
all_indices = df_features.index.tolist()

for i in range(0, len(all_indices), CHUNK_ROWS):
    chunk_idx = all_indices[i:i + CHUNK_ROWS]
    df_features.loc[chunk_idx, FEATURE_COLS] = scaler.transform(
        df_features.loc[chunk_idx, FEATURE_COLS].values
    ).astype("float32")
    gc.collect()

print("Scaler applied to full dataset.")
print(f"Shape: {df_features[['split', 'label'] + FEATURE_COLS].shape}")

Scaler applied to full dataset.
Shape: (1382402, 572)


## 4.4 - Confirm Normalization
Train split should have mean &approx; 0 and std &approx; 1 across all features. The test split will deviate substantially. It covers the attack period, a fundamentally different system state from normal operation. This is expected and documented.

In [17]:
for split in ["train", "test"]:
    split_data = df_features[df_features["split"] == split][FEATURE_COLS]
    mean_of_means = split_data.mean().mean()
    mean_of_stds  = split_data.std().mean()
    print(f"  {split:<6}  mean of feature means: {mean_of_means:>8.4f}  "
          f"mean of feature stds: {mean_of_stds:>8.4f}")

  train   mean of feature means:  -0.0000  mean of feature stds:   0.9468
  test    mean of feature means: 237.5340  mean of feature stds: 220.6748


## 4.5 - Save Scaler Artifact

In [18]:
scaler_path = REF_DIR / f"scaler_{RUN_ID}.joblib"
joblib.dump(scaler, scaler_path)

print(f"Scaler saved: {scaler_path}")
print(f"  Fitted on:  train split ({train_mask.sum():,} rows)")
print(f"  Features:   {len(FEATURE_COLS)}")

Scaler saved: work/wadi_A1/data/reference/scaler_20260223_142554_utc.joblib
  Fitted on:  train split (1,209,601 rows)
  Features:   570


# Stage 5 - Validate  
Confirms the feature matrix is structurally sound, leakage-free, and correctly distributed across splits.

## 5.1 - Checks

In [19]:
results = []

def record(check: str, passed: bool, detail: str = "") -> None:
    status = "PASS" if passed else "FAIL"
    results.append({"check": check, "status": status, "detail": detail})
    print(f"  [{status}] {check}" + (f" — {detail}" if detail else ""))

print("Structural checks:\n")

# S1: Feature count
print(f"Zero-variance sensor families confirmed dropped: {len(zero_std_base_sensors)} sensors, {len(zero_std_cols)} features\n")
record("S1 Feature count", len(FEATURE_COLS) == 570,
       f"expected 570, got {len(FEATURE_COLS)}")

# S2: No NaN in feature matrix
n_nan = df_features[FEATURE_COLS].isna().sum().sum()
record("S2 No NaN in feature matrix", n_nan == 0,
       f"{n_nan:,} NaN values found" if n_nan else "clean")

# S3: No inf in feature matrix
n_inf = np.isinf(df_features[FEATURE_COLS].values).sum()
record("S3 No inf in feature matrix", n_inf == 0,
       f"{n_inf:,} inf values found" if n_inf else "clean")

# S4: split and label columns present
for col in ["split", "label"]:
    record(f"S4 '{col}' column present", col in df_features.columns)

print("\nLeakage checks:\n")

# L1: Zero-variance sensor family fully removed
problem_sensors = {"3_AIT_001_PV", "2_LS_201_AH", "2_LS_401_AH"}
remaining = [c for c in FEATURE_COLS if c.split("__")[0] in problem_sensors]
record("L1 Zero-variance sensor families removed", len(remaining) == 0,
       f"still present: {remaining}" if remaining else f"{problem_sensors} and all variants removed")

# L2: Fault metadata columns absent
FAULT_META_COLS = ["fault_type", "fault_sensor", "fault_start",
                   "fault_end", "fault_severity"]
meta_present = [c for c in FAULT_META_COLS if c in df_features.columns]
record("L2 Fault metadata columns absent", len(meta_present) == 0,
       f"still present: {meta_present}" if meta_present else "all removed")

# L3: No future-looking rolling features
bad_rolling = [c for c in FEATURE_COLS if "center" in c.lower()]
record("L3 No center-aligned rolling features", len(bad_rolling) == 0,
       f"found: {bad_rolling}" if bad_rolling else "all windows backward-looking")

print("\nClass distribution checks:\n")

# C1: Expected labels present in each split
expected_labels = {
    "train": {0, 2},
    "test":  {1},
}
for split, exp in expected_labels.items():
    labels = set(df_features[df_features["split"] == split]["label"].unique())
    record(f"C1 Expected labels in {split}",
           exp.issubset({int(l) for l in labels}),
           f"found labels: {labels}")

print("\nCanary checks:\n")

# K1: Total row count
record("K1 Total row count", len(df_features) > 0,
       f"got {len(df_features):,}")

# K2: Train rows
actual_train = (df_features["split"] == "train").sum()
record("K2 Train row count", actual_train > 0,
       f"got {actual_train:,}")

# K3: Normalization sanity — train mean close to 0
train_mean = df_features[df_features["split"] == "train"][FEATURE_COLS].mean().mean()
record("K3 Train mean of feature means ≈ 0", abs(train_mean) < 0.01,
       f"mean={train_mean:.6f}")

# K4: Normalization sanity — train std close to 1
train_std = df_features[df_features["split"] == "train"][FEATURE_COLS].std().mean()
record("K4 Train mean of feature stds ≈ 1", abs(train_std - 1.0) < 0.10,
       f"std={train_std:.4f}")

print(f"\n{'='*60}")
passed = [r for r in results if r["status"] == "PASS"]
failed = [r for r in results if r["status"] == "FAIL"]
print(f"Validation summary: {len(passed)} passed, {len(failed)} failed")
if failed:
    print("\nFailed checks:")
    for r in failed:
        print(f"  {r['check']} — {r['detail']}")

Structural checks:

Zero-variance sensor families confirmed dropped: 3 sensors, 18 features

  [PASS] S1 Feature count — expected 570, got 570
  [PASS] S2 No NaN in feature matrix — clean
  [PASS] S3 No inf in feature matrix — clean
  [PASS] S4 'split' column present
  [PASS] S4 'label' column present

Leakage checks:

  [PASS] L1 Zero-variance sensor families removed — {'2_LS_201_AH', '2_LS_401_AH', '3_AIT_001_PV'} and all variants removed
  [PASS] L2 Fault metadata columns absent — all removed
  [PASS] L3 No center-aligned rolling features — all windows backward-looking

Class distribution checks:

  [PASS] C1 Expected labels in train — found labels: {np.int8(0), np.int8(2)}
  [PASS] C1 Expected labels in test — found labels: {np.int8(1)}

Canary checks:

  [PASS] K1 Total row count — got 1,382,402
  [PASS] K2 Train row count — got 1,209,601
  [PASS] K3 Train mean of feature means ≈ 0 — mean=-0.000000
  [PASS] K4 Train mean of feature stds ≈ 1 — std=0.9468

Validation summary: 14 pas

# Stage 6 - Write Outputs

## 6.1 - Assemble Final Output DataFrame

In [20]:
output_cols = ["timestamp", "split", "label"] + FEATURE_COLS
features_path = FEATURES_DIR / f"wadi_features_{RUN_ID}.parquet"
df_features[output_cols].to_parquet(features_path, index=False)

size_mb = features_path.stat().st_size / 1e6
print(f"Feature matrix written: {features_path}")
print(f"Size:                   {size_mb:.1f} MB")
print(f"Shape:                  ({len(df_features)}, {len(output_cols)})")

Feature matrix written: work/wadi_A1/data/features/wadi_features_20260223_142554_utc.parquet
Size:                   803.4 MB
Shape:                  (1382402, 573)


## 6.2 - Update `FEATURE_COLS` Reference with Final Counts

In [21]:
# Update FEATURE_COLS reference JSON with final counts
write_json(feature_cols_path, {
    "run_id":               RUN_ID,
    "dataset":              DATASET_NAME,
    "window_size":          WINDOW_SIZE,
    "min_periods":          MIN_PERIODS,
    "rolling_stats":        ROLLING_STATS,
    "n_raw_features":       len([c for c in FEATURE_COLS if "__" not in c]),
    "n_rolling_features":   len([c for c in FEATURE_COLS if "__" in c]),
    "n_total_features":     len(FEATURE_COLS),
    "zero_variance_dropped": zero_std_cols,
    "feature_cols":         FEATURE_COLS,
    "notes": [
        "Feature matrix excludes timestamp, observation_day, seconds_since_start.",
        "Rolling features computed per split independently — no cross-split leakage.",
        "NaN values filled with train split column means.",
        "3_AIT_001_PV dropped — all-zero in train split, zero variance, "
        "produces extreme scaled values in val/test.",
        "Scaler fit on train split only — applied to val and test.",
    ]
})

# Save scaler
scaler_path = REF_DIR / f"scaler_{RUN_ID}.joblib"
joblib.dump(scaler, scaler_path)

# Write run log
run_log = {
    "run_id":          RUN_ID,
    "created_at_utc":  utc_now_iso(),
    "stage":           "Notebook 4 — Feature Engineering",
    "dataset":         DATASET_NAME,
    "inputs": {
        "warehouse_parquet": str(wh_path),
        "sensor_cols_ref":   str(sensor_cols_path),
    },
    "outputs": {
        "feature_matrix":  str(features_path),
        "feature_cols_ref": str(feature_cols_path),
        "scaler":          str(scaler_path),
    },
    "configuration": {
        "window_size":   WINDOW_SIZE,
        "min_periods":   MIN_PERIODS,
        "rolling_stats": ROLLING_STATS,
    },
    "feature_summary": {
        "n_raw_features":     len(SENSOR_COLS) - 1,
        "n_rolling_features": len(rolling_cols) - 5,
        "n_total_features":   len(FEATURE_COLS),
        "zero_variance_dropped": zero_std_cols,
    },
    "dataset_summary": {
        "total_rows":  len(df_features),
        "total_cols":  len(output_cols),
        "label_counts": {
            int(k): int(v)
            for k, v in df_features["label"].value_counts().sort_index().items()
        },
    },
    "validation": {
        "passed": 14,
        "failed": 0,
    },
    "notes": [
        "3 zero-variance sensors dropped: 3_AIT_001_PV, 2_LS_201_AH, 2_LS_401_AH — 18 features removed total.",
    ],
}

run_log_path = RUN_DIR / f"run_{RUN_ID}.json"
write_json(run_log_path, run_log)
print(f"Feature cols ref updated: {feature_cols_path}")
print(f"Scaler saved:             {scaler_path}")
print(f"Run log written:          {run_log_path}")

Feature cols ref updated: work/wadi_A1/data/reference/feature_cols.json
Scaler saved:             work/wadi_A1/data/reference/scaler_20260223_142554_utc.joblib
Run log written:          work/wadi_A1/data/reference/pipeline_runs/run_20260223_142554_utc.json


# Stage 7 - Reflection
Documents key decisions, assumptions, and risks from this notebook.

In [22]:
reflection = [
    ("Feature matrix composition",
     f"{len(FEATURE_COLS)} features total: "
     f"{len([c for c in FEATURE_COLS if '__' not in c])} raw sensor readings + "
     f"{len([c for c in FEATURE_COLS if '__' in c])} rolling statistics. "
     f"{len(zero_std_base_sensors)} sensors dropped for zero variance in train split: "
     f"{', '.join(sorted(zero_std_base_sensors))}. "
     f"All rolling variants dropped with each sensor ({len(ROLLING_STATS) + 1} features per sensor)."),

    ("Rolling window design",
     f"Single {WINDOW_SIZE}-second backward-looking window applied per sensor "
     f"per split independently. Five statistics computed: mean, std, min, max, "
     f"rate of change. Window computed per split independently — no future data "
     f"crosses split boundaries. Min periods={MIN_PERIODS} (50% of window) "
     f"to handle split boundary NaNs cleanly."),

    ("NaN handling",
     "Two NaN sources addressed: rolling window warmup rows (first 30 rows "
     "per sensor per split) and pre-existing raw sensor NaNs from the warehouse "
     "parquet. Both filled with train split column means — consistent with "
     "normalization strategy, no leakage from val/test."),

    ("Zero-variance detection",
     "Sample-based scaler fit (100k train rows) used to detect zero-variance "
     "features before final normalization. Three sensors identified: "
     "3_AIT_001_PV (all-zero in train), 2_LS_201_AH and 2_LS_401_AH "
     "(binary level switches, near-zero variance in sample). All 18 features "
     "dropped before final scaler fit."),

    ("Normalization",
     "StandardScaler fit on train split only, applied to test. "
     "Closes the normalization leakage risk deferred from Notebook 3. "
     "Train mean≈0.0, std≈1.0. Test may deviate due to temporal distribution "
     "shift between normal operation and attack period — expected and documented."),

    ("Leakage audit — closed",
     "Both remaining open leakage risks from Notebook 3 are now closed: "
     "(1) normalization fit on train only — confirmed. "
     "(2) rolling windows backward-looking only — confirmed, no center=True."),

    ("Design decisions",
     "Single 60-second window chosen over multiple window sizes — all faults "
     "are at least 120 seconds long so the window always falls within the fault "
     "signature. Per-sensor features only — cross-sensor features noted as a "
     "natural extension for future work. Both decisions documented for the "
     "methodology section."),

    ("Next step",
     "Notebook 5 — Model Training. Loads wadi_features_*.parquet, trains "
     "Logistic Regression baseline and Random Forest primary model on train "
     "split, and produces final test set evaluation with per-class precision, "
     "recall, and F1. No validation split — threshold selection performed "
     "analytically on train or reported as a sweep over the test set."),
]

print("Pipeline Reflection")
print("=" * 60)
for title, content in reflection:
    print(f"\n[{title}]")
    print(f"  {content}")

Pipeline Reflection

[Feature matrix composition]
  570 features total: 95 raw sensor readings + 475 rolling statistics. 3 sensors dropped for zero variance in train split: 2_LS_201_AH, 2_LS_401_AH, 3_AIT_001_PV. All rolling variants dropped with each sensor (6 features per sensor).

[Rolling window design]
  Single 60-second backward-looking window applied per sensor per split independently. Five statistics computed: mean, std, min, max, rate of change. Window computed per split independently — no future data crosses split boundaries. Min periods=30 (50% of window) to handle split boundary NaNs cleanly.

[NaN handling]
  Two NaN sources addressed: rolling window warmup rows (first 30 rows per sensor per split) and pre-existing raw sensor NaNs from the warehouse parquet. Both filled with train split column means — consistent with normalization strategy, no leakage from val/test.

[Zero-variance detection]
  Sample-based scaler fit (100k train rows) used to detect zero-variance features

# Continues with Notebook 5 - Binary Classification (Normal vs Anomaly)